In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes

Solving environment: / 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2, defaults::certifi-2019.9.11-py37_0
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0
  - anaconda::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - defaults::ca-certificates-2019.8.28-0, defaults::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, anaconda::certifi-2019.9.11-py37_0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::certifi-2019.9.11-py37_0, defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

# All requested packages already installed.

Solving 

In [45]:
# Import Pandas and Numpy
import pandas as pd
import numpy as np
import requests

#import dotenv for managing secrets
import os
from dotenv import load_dotenv

# Import Geocoding
from geopy.geocoders import Nominatim

# Import Matplotlib and associated plotting modules
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Import Folium for Visualisation
import folium # map rendering library

In [46]:
load_dotenv("config.txt")
fs_version = os.environ.get("VERSION")
fs_clientid = os.environ.get("CLIENT_ID")
fs_secret = os.environ.get("CLIENT_SECRET")
fs_limit = 30

In [47]:
# Load the dataset we saved previously in part 2
df = pd.read_csv("part2.csv")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [48]:
# Get the Geocode Address of Toronto, Canada
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
lat = location.latitude
lng = location.longitude
print("Toronto: Lat {} Lng {}".format(lat,lng))

GeocoderTimedOut: Service timed out

In [ ]:
# Create Map of Toronto
toronto_map = folium.Map(location=[lat,lng], zoom_start=10)

# Add Markers for all the neighbourhood data
for lat,lng,neighborhood, borough in zip(df["Latitude"],df["Longitude"],df["Neighborhood"], df["Borough"]):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="blue",
        fill_color="blue",
        fill=True,
        fill_opacity=0.5
    ).add_to(toronto_map)

toronto_map

In [49]:
# Only work with Toronto Boroughs
df_central = df[df["Borough"].str.contains("Toronto")]
df_central
df_central.shape


(39, 5)

In [50]:
map_central = folium.Map([lat,lng], zoom_start=10)

for lat,lng,neighborhood,borough in zip(df_central["Latitude"], df_central["Longitude"], df_central["Neighborhood"], df_central["Borough"]):
    label = "{}, {}".format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color="yellow",
        fill=True,
        fill_color="black",
        fill_opacity=0.5,
    ).add_to(map_central)

map_central

In [51]:
# Explore the nearby venues for each of out neighbourhoods
# Lets see how many neighbourhoods are in our reduced dataframe
df_central.shape

(39, 5)

In [52]:
df_central["Borough"].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [53]:
df_central["Neighborhood"].value_counts()

Stn A PO Boxes 25 The Esplanade                                                                         1
Studio District                                                                                         1
Moore Park,Summerhill East                                                                              1
Lawrence Park                                                                                           1
Business Reply Mail Processing Centre 969 Eastern                                                       1
Harbourfront                                                                                            1
The Danforth West,Riverdale                                                                             1
Roselawn                                                                                                1
Brockton,Exhibition Place,Parkdale Village                                                              1
Christie                                      

In [95]:
def get_nearby(names, lats, lngs, radius=500, limit=fs_limit):
    
    # Set Foursquare Authentication Credentials
    tkn = "client_id={}&client_secret={}&v={}".format(fs_clientid, fs_secret, fs_version)
    venue_list = []
    
    for name, lat, lng in zip(names, lats, lngs):
        # Set the API call
        api = "https://api.foursquare.com/v2/venues/explore?ll={},{}&radius={}&limit={}&{}".format(lat,lng,radius,limit,tkn)
        print("Getting Data for {}".format(name))
        
        # Make the API call
        result = requests.get(api).json()
        
        # Check some valid data was returned
        if len(result["response"]["groups"]) > 0:
            venues = result["response"]["groups"][0]["items"]

            for venue in venues:
                venue_list.append([(name, lat, lng, 
                                  venue["venue"]["name"], 
                                  venue["venue"]["location"]["lat"],
                                  venue["venue"]["location"]["lng"],
                                  venue["venue"]["categories"][0]["name"])])
            
        else:
            print("No results returned for {}".format(name))
    
    nearby = pd.DataFrame([item for row in venue_list for item in row])
    nearby.columns = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    return nearby


In [97]:
df_near = get_nearby(df_central["Neighborhood"], df_central["Latitude"], df_central["Longitude"])
df_near

Getting Data for The Beaches
Getting Data for The Danforth West,Riverdale
Getting Data for The Beaches West,India Bazaar
Getting Data for Studio District
Getting Data for Lawrence Park
Getting Data for Davisville North
Getting Data for North Toronto West
Getting Data for Davisville
Getting Data for Moore Park,Summerhill East
Getting Data for Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Getting Data for Rosedale
Getting Data for Cabbagetown,St. James Town
Getting Data for Church and Wellesley
Getting Data for Harbourfront
Getting Data for Ryerson,Garden District
Getting Data for St. James Town
Getting Data for Berczy Park
Getting Data for Central Bay Street
Getting Data for Adelaide,King,Richmond
Getting Data for Harbourfront East,Toronto Islands,Union Station
Getting Data for Design Exchange,Toronto Dominion Centre
Getting Data for Commerce Court,Victoria Hotel
Getting Data for Roselawn
Getting Data for Forest Hill North,Forest Hill West
Getting Data for The Annex,Nort

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
...,...,...,...,...,...,...,...
861,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
862,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
863,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
864,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station


In [98]:
# Check how many rows were fetched
df_near.shape

(866, 7)

In [100]:
# Examine the count of nearby venues per neighbourhood
df_near.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",18,18,18,18,18,18
"Cabbagetown,St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown,Grange Park,Kensington Market",30,30,30,30,30,30
Christie,16,16,16,16,16,16


In [103]:
# Check how many category types were returned
df_near["Venue Category"].value_counts()

Coffee Shop              65
Café                     61
Park                     29
Italian Restaurant       25
Restaurant               24
                         ..
College Arts Building     1
Food Court                1
Poke Place                1
Falafel Restaurant        1
Dog Run                   1
Name: Venue Category, Length: 190, dtype: int64